In [1]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.0.185.138;DATABASE=JYDB;UID=js_dev;PWD=js_dev')

In [16]:
query = 'SELECT t.SecuCode, t.SecuAbbr, pf.InfoPublDate, pf.ForcastType, pf.ForcastContent \
         FROM LC_PerformanceForecast as pf, SecuMain as t \
         WHERE pf.CompanyCode = t.CompanyCode \
             AND t.SecuCategory = 1 \
             AND pf.ForecastObject = 10 \
             ORDER by pf.InfoPublDate DESC'

In [17]:
df = pd.read_sql_query(query, con=conn)

In [18]:
df.shape

(89164, 5)

# 提取10-1月份预告

In [22]:
df['Month'] = df['InfoPublDate'].apply(lambda x: x.month)

In [32]:
df = df[df['Month'].isin([10,11,12,1])]

In [55]:
df.tail()

,SecuCode,SecuAbbr,InfoPublDate,ForcastType,ForcastContent,Month,Year
88183,600231,凌钢股份,2002-10-15,3,四季有望继续取得较好业绩,10,2002
88184,600821,津 劝 业,2002-10-15,1,第三季度利润将会出现亏损,10,2002
88185,600011,华能国际,2002-10-10,4,前三季度发电量再次增长,10,2002
88186,600053,九鼎投资,2002-10-09,1,第三季度业绩将会出现亏损,10,2002
89163,000005,世纪星源,2002-01-30,1,世纪星源董事会公告：因相关主营业务发展进度推迟，2001年度主营业务收入未能达致预期...,1,2002


# 按年统计

In [35]:
df['Year'] = df['InfoPublDate'].apply(lambda x: x.year)

C:\Users\zhangyang01\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
count_df = pd.DataFrame(0, index=range(2001, 2019), columns=range(1,11)+range(18,23))
for i in df.index:
    if df.loc[i, 'Month'] in [10,11,12]:
        count_df.loc[df.loc[i, 'Year'], df.loc[i, 'ForcastType']] += 1
    elif df.loc[i, 'Month'] == 1:
        count_df.loc[df.loc[i, 'Year']-1, df.loc[i, 'ForcastType']] += 1

In [66]:
count_df

,1,2,3,4,5,6,7,8,9,10,18,19,20,21,22
2001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2002,226,51,130,115,19,16,1,118,0,0,0,0,0,0,0
2003,279,25,107,230,2,6,5,91,0,0,0,0,0,0,0
2004,314,13,125,344,7,5,4,110,4,0,0,0,0,0,0
2005,415,12,106,201,0,8,6,150,5,0,0,0,0,0,0
2006,280,10,199,390,2,0,5,84,0,0,0,0,0,0,0
2007,225,12,182,803,12,9,0,72,0,0,0,0,0,0,0
2008,388,12,122,485,39,4,0,397,10,0,0,0,0,0,0
2009,367,8,249,680,69,1,6,226,1,0,0,0,0,0,0
2010,252,16,221,1025,88,1,7,190,0,0,2,0,0,0,0


In [67]:
count_df.columns = [u'预亏', u'预警', u'预盈', u'预增', u'预平', u'经营计划', u'减亏', u'预降', u'减增',
                    u'提前披露', u'减降', u'由盈转亏', u'扭亏为盈', u'同向上升', u'同向下降']

In [68]:
pct_df = count_df.div(count_df.sum(axis=1), axis=0)

In [69]:
pct_df.to_excel(u'C:/Users/zhangyang01/Desktop/业绩预告统计.xlsx')

In [70]:
count_df.to_excel(u'C:/Users/zhangyang01/Desktop/统计.xlsx')